# Data Collection

In [4]:
import cv2
import os

# Load Haar cascade
face_cascade = cv2.CascadeClassifier(r'C:\Users\Md Fazlul Bary\OneDrive - American International University-Bangladesh\Artificial Inteligence\MID_Project_CVPR\haarcascade_frontalface_default.xml')

# Setup webcam
cap = cv2.VideoCapture(0)

# Get the person's name
person_name = input("Enter the person's name: ")
save_path = f'faces/{person_name}'
os.makedirs(save_path, exist_ok=True)

# Capture images
count = 0
while count < 500: 
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  #frame is converted to grayscale
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (128, 128))
        cv2.imwrite(f"{save_path}/{count}.jpg", face_resized)
        count += 1

    cv2.imshow("Capturing Faces", frame)
    if cv2.waitKey(1) & 0xFF == ord('q') or count >= 500:
        break

cap.release()
cv2.destroyAllWindows()
print(f"Captured {count} images for {person_name}.")

Enter the person's name: shanto
Captured 475 images for shanto.


# Build CNN Model

In [5]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the dataset
data = []
labels = []
label_map = {}

base_path = 'faces'
for i, person in enumerate(os.listdir(base_path)):
    label_map[i] = person
    person_path = os.path.join(base_path, person)
    for img_name in os.listdir(person_path):
        img_path = os.path.join(person_path, img_name)
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, (128, 128)) / 255.0
        data.append(img_resized)
        labels.append(i)

data = np.array(data)
labels = np.array(labels)

# Build CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(label_map), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(data, labels, epochs=10, validation_split=0.2)
model.save("face_recognition_model.h5")

# Save label map
with open("label_map.txt", "w") as f:
    f.write(str(label_map))

print("Model trained and saved successfully!")


E:\Anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 23s 263ms/step - accuracy: 0.7671 - loss: 0.7719 - val_accuracy: 0.1597 - val_loss: 12.6061
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 18s 235ms/step - accuracy: 0.9792 - loss: 0.0684 - val_accuracy: 0.1597 - val_loss: 14.0181
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 18s 234ms/step - accuracy: 0.9990 - loss: 0.0074 - val_accuracy: 0.1597 - val_loss: 15.3325
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 17s 231ms/step - accuracy: 0.9971 - loss: 0.0389 - val_accuracy: 0.1597 - val_loss: 13.8304
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 19s 247ms/step - accuracy: 0.9962 - loss: 0.0090 - val_accuracy: 0.1597 - val_loss: 15.6614
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 23s 306ms/step - accuracy: 0.9998 - loss: 5.5198e-04 - val_accuracy: 0.1597 - val_loss: 15.4059
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 20s 267ms/step - accuracy: 0.9988 - loss: 0.0021 - val_accuracy: 0.1597 - val_loss: 16.6130
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 23s 303ms/step - accuracy: 0.9969 - loss: 0.0100

Model trained and saved successfully!


In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,179,284 (84.61 MB)

 Trainable params: 7,393,094 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 14,786,190 (56.40 MB)

# Face Recognization and Taking Attendence 

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import pandas as pd
import datetime
import os
import time

# Load Haar cascade and model
face_cascade = cv2.CascadeClassifier(r'C:\Users\Md Fazlul Bary\OneDrive - American International University-Bangladesh\Artificial Inteligence\MID_Project_CVPR\haarcascade_frontalface_default.xml')
model = load_model(r"C:\Users\Md Fazlul Bary\OneDrive - American International University-Bangladesh\Artificial Inteligence\MID_Project_CVPR\face_recognition_model.h5")

# Load label map
with open("label_map.txt", "r") as f:
    label_map = eval(f.read())

# Attendance file
attendance_file = (r"C:\Users\Md Fazlul Bary\OneDrive - American International University-Bangladesh\Artificial Inteligence\MID_Project_CVPR\attendance.xlsx")
if not os.path.exists(attendance_file):
    pd.DataFrame(columns=["Name", "Date", "Time"]).to_excel(attendance_file, index=False)

# Start webcam
cap = cv2.VideoCapture(0)

# Variable to store the last time attendance was marked
last_attendance_time = None
attendance_interval = 5  # Minimum time between entries in seconds

# Counter to track if attendance has been marked
attendance_marked = {}

# Variable to store the attendance message
attendance_message = ""
message_time = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (128, 128)) / 255.0
        face_resized = np.expand_dims(face_resized, axis=0)

        predictions = model.predict(face_resized)
        predicted_label = np.argmax(predictions)
        name = label_map[predicted_label]

        # Get the current time
        now = datetime.datetime.now()

        # Check if enough time has passed since the last entry
        if last_attendance_time is None or (now - last_attendance_time).seconds >= attendance_interval:
            # Check if attendance for the person has already been marked today
            df = pd.read_excel(attendance_file)
            if not ((df["Name"] == name) & (df["Date"] == now.date())).any():
                new_entry = pd.DataFrame({"Name": [name], "Date": [now.date()], "Time": [now.time()]})
                df = pd.concat([df, new_entry], ignore_index=True)
                df.to_excel(attendance_file, index=False)
                print(f"Attendance: {name}.")

                # Update the last attendance time
                last_attendance_time = now

                # Store the attendance message
                attendance_message = f"Attendance: {name} at {now.strftime('%Y-%m-%d %H:%M:%S')}"

                message_time = time.time()  # Record the time the message was shown

        # Display name on the screen above the face
        cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1)


        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # If an attendance message exists and enough time has passed, display it on the screen
    if attendance_message and (time.time() - message_time) < 5:  # Show message for 5 seconds
        cv2.putText(frame, attendance_message, (5, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)

    # Display the frame with annotated faces
    cv2.imshow("Face Recognition Attendance", frame)

    # Check if 'q' is pressed to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
Attendance: Shawon.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Attendance: shanto.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━